# CESCO Q&A를 통한 가정집 해충 조사

* 세스코 홈페이지에 등록된 26330개 QnA의 데이터 수집 및 분석
* 홈페이지가 Ajax를 사용하기 때문에 GET방식의 호출이 불가능하여 Form 데이터 확인 후 POST방식의 requests 호출

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import platform

In [7]:
driver = None
if platform.platform() == 'Windows':
    driver = webdriver.Chrome('chromedriver.exe')
elif platform.platform() == 'Linux' or platform.platform() == 'Darwin':
    driver = webdriver.Chrome('chromedriver')
else:
    # chrome drivers: https://chromedriver.storage.googleapis.com/index.html
    raise Exception('운영체제에 맞는 Chrome 드라이버를 다운로드 후 압축을 푼 뒤, 소스코드와 같은 디렉토리에 실행파일을 넣어준 뒤 실행해주세요')

C:\Users\image\AppData\Local\Temp/ipykernel_25164/1732518312.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [3]:
req = requests.get('https://cesco.co.kr/Cesco/support/support_qna_list.aspx?Page=1&sSdate=&sEdate=&sType=&sField=1&sString=&sGbn=')

In [5]:
soup = BeautifulSoup(req.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <!-- 2020-06-01 -->
  <script charset="UTF-8" src="//t1.daumcdn.net/adfit/static/kp.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   kakaoPixel('4698903025612243960').pageView();
  </script>
  <!-- Facebook Pixel Code -->
  <script>
   !function(f,b,e,v,n,t,s)
{if(f.fbq)return;n=f.fbq=function(){n.callMethod?
n.callMethod.apply(n,arguments):n.queue.push(arguments)};
if(!f._fbq)f._fbq=n;n.push=n;n.loaded=!0;n.version='2.0';
n.queue=[];t=b.createElement(e);t.async=!0;
t.src=v;s=b.getElementsByTagName(e)[0];
s.parentNode.insertBefore(t,s)}(window,document,'script',
'https://connect.facebook.net/en_US/fbevents.js');
fbq('init', '3155073957846647'); 
fbq('track', 'PageView');
  </script>
  <noscript>
   <img height="1" src="https://www.facebook.com/tr?id=3155073957846647&amp;ev=PageView
&amp;noscript=1" width="1"/>
  </noscript>
  <!-- End Facebook Pixel Code -->
  <!-- Google Tag Man